## LLM with Gradio

Earlier we learned how to call different LLMs from python code. We also learned how to build UI application with very little coding using Gradio framework. Today we will combine those two together, by building simple Gen AI application with UI and perform various generative tasks by utilizing LLM models.

### Plan
In this exercise, we are going to build the following -

* Connect UI with the LLM
    * [Simple Q&A application](#Simple-q&a-application)
    * [Q&A app with markdown response](#Q&A-application-with-markdown-response)
    * [Q&A app with LLM of choice](#Q&A-application-with-llm-of-choice)
    * [Simple chatbot application](#Simple-chatbot-application)
    * [Simple chatbot with streamed output](#Simple-chatbot-with-streamed-output)
    * [Chatbot with tool Calling](#Chatbot-with-tool-Calling)
    * [Chatbot with tool calling (Enhanced Version)](#Chatbot-with-tool-calling-Enhanced-Version)
    * Handle multiple tools call from AI assistant


### Implementation - to be carried out through the following steps.

#### Import necessary libraries

In [3]:
# To load environment variables
import os
from dotenv import load_dotenv 

# To call the OpenAI library which can call any commercial or open-source LLMs
from openai import OpenAI

# To display the output in nice format instead of plain text
from IPython.display import Markdown, display

# To scrape the content from a website
from bs4 import BeautifulSoup
import requests

# To build the UI
import gradio as gr

# To parse the JSON response from LLM (which we need for tool calling in chatbot)
import json


#### Get the API Key from the environment variable

In [2]:
load_dotenv(override=True)

# OpenAI
OPENAI_BASE_URL = "https://api.openai.com/v1" # This is the default base url for OpenAI library
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not OPENAI_API_KEY.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif OPENAI_API_KEY.strip() != OPENAI_API_KEY:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("OpenAI API key found and looks good so far!")

# Google
GOOGLE_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Validate the Google API key
if not GOOGLE_API_KEY:
    print("No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini")
elif not GOOGLE_API_KEY.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("Google API key found and looks good so far!")

# OpenRouter
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

# Validate the OpenRouter API key
if not OPENROUTER_API_KEY:
    print("No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini")
else:
    print("OpenRouter API key found and looks good so far!")


# Ollama running locally
OLLAMA_BASE_URL = "http://localhost:11434/v1" 
OLLAMA_API_KEY = "NA" # the api_key is not relevant as the model is running locally

OpenAI API key found and looks good so far!
Google API key found and looks good so far!
OpenRouter API key found and looks good so far!


#### Simple Q&A application


In [ ]:
# Define system prompt
system_prompt = "You are a helpful assistant, always answer in a courteous manner."

# Define the custom function
def first_chatbot(user_prompt):
    # Instantiate OpenAI with proper API key
    openai = OpenAI(api_key=OPENAI_API_KEY)

    # Call the LLM with the system and user prompt
    response = openai.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    # Return the output generated by the LLM
    return response.choices[0].message.content


# Instantiate gradio interface
interface = gr.Interface(
    fn=first_chatbot,
    title="First Chatbot",
    inputs=gr.Textbox(label="User prompt"),
    outputs=gr.Textbox(label="Response from LLM", lines=10)
)

interface.launch()

#### Q&A application with markdown response

In [ ]:
# Define system prompt
system_prompt = "You are a helpful assistant, always answer in a courteous manner."

# Define the custom function
def first_chatbot(user_prompt):
    # Instantiate OpenAI with proper API key
    openai = OpenAI(api_key=OPENAI_API_KEY)

    # Call the LLM with the system and user prompt
    response = openai.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    # Return the output generated by the LLM
    return response.choices[0].message.content


# Instantiate gradio interface
interface = gr.Interface(
    fn=first_chatbot,
    title="First Chatbot",
    inputs=gr.Textbox(label="User prompt", lines=5),
    outputs=gr.Markdown(label="Response from LLM") # This is the only change from the previous example
)

interface.launch()

#### Q&A application with LLM of choice

In [ ]:
# Define system prompt
system_prompt = """
You are an expert in latest technologies. 
Provide detailed explanation. 
Do not assume user has any prior knowledge.
"""

# Define the custom function
def chatbot_with_llm_of_choice(user_prompt, user_model):
    # Instantiate OpenAI with proper API key
    if user_model == "GPT from OpenAI":
        model_api = OpenAI(api_key=OPENAI_API_KEY, base_url=OPENAI_BASE_URL)
        model="gpt-4.1-nano"
    elif user_model == "Gemini from Google":
        model_api = OpenAI(api_key=GOOGLE_API_KEY, base_url=GOOGLE_BASE_URL)
        model="gemini-2.5-flash-lite"
    elif user_model == "Mistral Devstral via OpenRouter":
        model_api = OpenAI(api_key=OPENROUTER_API_KEY, base_url=OPENROUTER_BASE_URL)
        model="mistralai/devstral-2512:free"
    elif user_model == "Deepseek from Ollama at Local":
        model_api = OpenAI(api_key=OLLAMA_API_KEY, base_url=OLLAMA_BASE_URL)
        model="deepseek-r1:1.5b"
    else:
        return "Invalid model"

    # Call the LLM with the system and user prompt
    response = model_api.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    # Return the output generated by the LLM
    return response.choices[0].message.content


# Instantiate gradio interface
interface = gr.Interface(
    fn=chatbot_with_llm_of_choice,
    title="Chatbot with LLM of Choice",
    inputs=[
        gr.Textbox(label="User prompt", lines=5),
        gr.Dropdown(
            label="Select LLM", 
            value="GPT from OpenAI",
            choices=[
                "GPT from OpenAI", 
                "Gemini from Google",
                "Mistral Devstral via OpenRouter",
                "Deepseek from Ollama at Local"
            ] 
        )
    ],
    outputs=gr.Markdown(label="Response from LLM")
)

interface.launch()

#### Simple chatbot application

So far we have built Q&A application and ran few use cases, where user can enter their question on the left, and the LLM generates answer on the right. Now, we are going to build a chatbot, which will be like a flow of conversation between question from user and response from LLM.  

In [ ]:
# Define system prompt
system_prompt = "You are a helpful assistant, always answer in a courteous manner."

# Define the custom function
def first_chatbot(user_prompt, context_history):
    
    # Instantiate OpenAI with proper API key
    model_api = OpenAI(api_key=OPENROUTER_API_KEY, base_url=OPENROUTER_BASE_URL)
    model="mistralai/devstral-2512:free"

    # Extract 'role' and 'context' values from the conversation history and build a dictionary
    history = [{"role":h["role"], "content":h["content"]} for h in context_history]
    # Create the new message list with system prompt, conversation history and the new user prompt
    messages=[{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": user_prompt}]

    # Call the LLM with the system and user prompt
    response = model_api.chat.completions.create(
        model=model,
        messages=messages
    )

    # Return the output generated by the LLM
    return response.choices[0].message.content


# Instantiate gradio interface
interface = gr.ChatInterface(fn=first_chatbot, type="messages", title="First ChatBot")

interface.launch()

The Chatbot above with the free mistralai model via OpenRouter did great. Where I started with my first question *"What are the top 10 companies in the world in terms of revenue?"* and it gave the correct answer. Then I asked *"How do these companies rank in terms of profit?"* and the LLM responded with the profit data and comparison between the same 10 companies from the answer of the first question. If we didn't feed the context history then the model would have produced top companies in terms of profit. 

#### Simple chatbot with streamed output

We will build very similar chatbot as above but add the streaming effect, so the user will see a stream of text as the LLM is generating output, instead of waiting with a blank screen till the model finished generating the entire content of the output.

In [ ]:
# Define system prompt
from email import message


system_prompt = "You are a helpful assistant, always answer in courteous manner and in simple language"

# Define the custom function
def first_chatbot(user_prompt, context_history):
    
    # Instantiate OpenAI with proper API key
    model_api = OpenAI(api_key=OPENAI_API_KEY, base_url=OPENAI_BASE_URL)
    model="gpt-4.1-mini"

    # Extract 'role' and 'context' values from the conversation history and build a dictionary
    history = [{"role":h["role"], "content":h["content"]} for h in context_history]
    # Create the new message list with system prompt, conversation history and the new user prompt
    messages=[{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": user_prompt}]

    # Call the LLM with the combined prompt
    stream = model_api.chat.completions.create(
        model=model,
        messages=messages,
        stream=True # Change from previous example, this makes the API to return token incrementally
    )

    # Capture the incremental response and send to the UI
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or '' # instead of message.content we use delta.content to get the change
        yield response # the 'yield' sends the incremental response to Gradio, unlike 'return' which sends the final response at the end



# Instantiate gradio interface
interface = gr.ChatInterface(fn=first_chatbot, type="messages", title="First ChatBot")

interface.launch()

#### Interesting Observation

I first tried the chatbot above with streamed output by using the *mistralai/devstral-2512:free* model from *OpenRouter*. However that ran into error while generating content. Then I switched to *gpt-4.1-mini* model and the chatbot worked perfectly fine with multiple prompts.

#### Chatbot with tool calling

We have built the regular chatbot functionality where LLM answers to user prompt. Now, along with that, we will let LLM to decide whether there is a need to call a tool (which will be a simple function to begin with). Then the UI (Gradio) will send the response from the LLM indicating the need for a tool (the custom function) call to the backend code. The backend code will parse the response, call the tool (the custom function), and send the response along with the output generated by the custom function (tool). Then the LLM will parse that message and respond to the user.

So, there will be some additional steps when we need to code a chatbot with tool calling capability, over simple chatbots which we have built earlier. Those steps are as follows -
1. Define the tool, which is a custom function to perform any task we would like.
2. Create a JSON in a certain way, which will tell the LLM about what tool needs to be called in which condition.
3. Pass that JSON as a new parameter called *tool* while calling the LLMs *chat.completions.create* function
4. Create a new function, which will 
    * Parse the response from LLM, 
    * Evaluate if LLM indicated any tool call
    * If so then will call the custom function (tool)
    * Create the output in specific format
    * We can customize this function if we may need to call different custom functions (tools) in future.
5. Combine the response from the new function and the original LLM response and send that to the LLM for second time. 

In [ ]:
# Define system prompt
system_prompt = """
You are a friendly travel booking assistant. Please answer in one sentence. 
For any non travel related question please respond saying that
Sorry I do not have the information, However I can help you related to travel booking
"""

# *** Begin - Custom Funtion (Tool) ***

# Dictionary with ticket price to different cities - we can modify this to find from a DB table
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

# Function to get the ticket price of a given city
def get_ticket_price(destination_city):
    # The following print statement is optional to show when tool calling is happening in code output
    print(f"Tool called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknown ticket price")
    return f"The price of a ticket to {destination_city} is {price}"

# *** End - Custom Funtion (Tool) ***


# *** Begin - Create a JSON, which will tell the LLM about what tool needs to be called in which condition.***

price_function = {
    "name": "get_ticket_price", # The name of the custom function (tool)
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": { # Define each parameters required by the tool function, we have only 1 called destination_city
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string", # Data type of the input parameter
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": price_function}]
# *** End - Create a JSON, which will tell the LLM about what tool needs to be called in which condition.***


# *** Begin - The function that parses the response fro first LLM call, call the tool function and get response from it

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        # Parse the arguments for the tool function from the message
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        # Call the tool function
        price_details = get_ticket_price(city)
        # Generate the response in following JSON format
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.id
        }
    return response

# *** End - The function that parses the response fro first LLM call, call the tool function and get response from it

# Define the chat function
def chatbot_with_tool(user_prompt, context_history):
    
    # Instantiate OpenAI with proper API key
    model_api = OpenAI(api_key=OPENAI_API_KEY, base_url=OPENAI_BASE_URL)
    model="gpt-4.1-mini"

    # Extract 'role' and 'context' values from the conversation history and build a dictionary
    history = [{"role":h["role"], "content":h["content"]} for h in context_history]
    # Create the new message list with system prompt, conversation history and the new user prompt
    messages=[{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": user_prompt}]

    # Make the FIRST call to the LLM with the system prompt, user prompt, and the chat history
    response = model_api.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools # This is the new parameter with all the info of custom function (tool) to the LLM
    )

    # Evaluate if the LLM response indicated any need of tool call
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        tool_response = handle_tool_call(message)
        messages.append(message)
        messages.append(tool_response)

        # Make the SECOND call to the LLM with the entire message thread combined with the response from the tool
        # Note that this time we are not passing the third parameter 'tools=tools', because we just want the LLM
        # to summarise the entire message and the response from tool and generate a nice output to the user.
        response = model_api.chat.completions.create(model=model, messages=messages)
    else:
        # The following print statement is optional to show when tool calling is happening in code output
        print("No tool called yet")


    # Return the output generated by the LLM
    return response.choices[0].message.content


# Instantiate gradio interface
interface = gr.ChatInterface(fn=chatbot_with_tool, type="messages", title="ChatBot with Tool Calling")

interface.launch()

#### Chatbot with tool calling (Enhanced Version)

We have built the chatbot functionality with ability to call custom function or tool. However, when we try to call the tool more than once, such as for prompt like 'I am thinking to go to either London or Paris, which option will be cheaper'. Here the tool needs to be called twice, once to get the ticket price for London and then again for Paris. In such case the above code will run into error, as it's created in a way to call the tool only once. 

We are going to fix the issue by making a simple change in the custom function - *handle_tool_call*

In [ ]:
# Define system prompt
system_prompt = """
You are a friendly travel booking assistant. Please answer in one sentence. 
For any non travel related question please respond saying that
Sorry I do not have the information, However I can help you related to travel booking
"""

# *** Begin - Custom Funtion (Tool) ***

# Dictionary with ticket price to different cities - we can modify this to find from a DB table
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

# Function to get the ticket price of a given city
def get_ticket_price(destination_city):
    # The following print statement is optional to show when tool calling is happening in code output
    print(f"Tool called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknown ticket price")
    return f"The price of a ticket to {destination_city} is {price}"

# *** End - Custom Funtion (Tool) ***


# *** Begin - Create a JSON, which will tell the LLM about what tool needs to be called in which condition.***

price_function = {
    "name": "get_ticket_price", # The name of the custom function (tool)
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": { # Define each parameters required by the tool function, we have only 1 called destination_city
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string", # Data type of the input parameter
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": price_function}]
# *** End - Create a JSON, which will tell the LLM about what tool needs to be called in which condition.***


# *** Begin - The function that parses the response fro first LLM call, call the tool function and get response from it

def handle_tool_call_adv(message):
    response = [] # Define an empty list

    for tool_call in message.tool_calls: # This is the change from above to loop through to handle multiple tool calls
        if tool_call.function.name == "get_ticket_price":
            # Parse the arguments for the tool function from the message
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            # Call the tool function
            price_details = get_ticket_price(city)
            # Generate the response in the following JSON format and append to the list
            response.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
    return response

# *** End - The function that parses the response fro first LLM call, call the tool function and get response from it

# Define the chat function
def chatbot_with_tool_adv(user_prompt, context_history):
    
    # Instantiate OpenAI with proper API key
    model_api = OpenAI(api_key=OPENAI_API_KEY, base_url=OPENAI_BASE_URL)
    model="gpt-4.1-mini"

    # Extract 'role' and 'context' values from the conversation history and build a dictionary
    history = [{"role":h["role"], "content":h["content"]} for h in context_history]
    # Create the new message list with system prompt, conversation history and the new user prompt
    messages=[{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": user_prompt}]

    # Make the FIRST call to the LLM with the system prompt, user prompt, and the chat history
    response = model_api.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools # This is the new parameter with all the info of custom function (tool) to the LLM
    )

    # Evaluate if the LLM response indicated any need of tool call
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        tool_responses = handle_tool_call_adv(message)
        messages.append(message)
        # Instead of 'append', we use 'extend' to add each element from tool response separately
        # This is essential to allow tool calls more than once in a single prompt. 
        # messages.append(tool_response)
        messages.extend(tool_responses)

        # Make the SECOND call to the LLM with the entire message thread combined with the response from the tool
        # Note that this time we are not passing the third parameter 'tools=tools', because we just want the LLM
        # to summarise the entire message and the response from tool and generate a nice output to the user.
        response = model_api.chat.completions.create(model=model, messages=messages)
    else:
        # The following print statement is optional to show when tool calling is happening in code output
        print("No tool called yet")


    # Return the output generated by the LLM
    return response.choices[0].message.content


# Instantiate gradio interface
interface = gr.ChatInterface(fn=chatbot_with_tool_adv, type="messages", title="ChatBot with Tool Calling")

interface.launch()

#### Chatbot with multiple tool calling

We have built the chatbot functionality with ability to call custom function or tool. Now we will make couple of modifications, such as -
* We are going to add multiple custom functions or tools for LLM to call as needed based on user prompt.
* We will do some basic database operations in the tools functions.

So, we are going to make some modifications in the code we wrote above, such as -
1. Define the custom function for each tool with necessary database operation.
2. Create JSON dictionary for each tool for LLM to understand.
3. Modify the custom function, which will 
    * Parse the response from LLM.
    * Evaluate if LLM indicated any tool call.
    * Decide which tool to call based on the response.
    * If so then will call the custom function (tool).
    * Create the output in specific format.
4. Combine the response from the new function and the original LLM response and send that to the LLM. 

In [ ]:
# Define system prompt
system_prompt = """
You are a friendly travel booking assistant. Please answer in one sentence. 
For any non travel related question please respond saying that
Sorry I do not have the information, However I can help you related to travel booking.
"""

# *** Begin - First Custom Funtion (Tool) ***

# Dictionary with ticket price to different cities - we can modify this to find from a DB table
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

# Function to get the ticket price of a given city
def get_ticket_price(destination_city):
    # The following print statement is optional to show when tool calling is happening in code output
    print(f"Price check tool called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknown ticket price")
    return f"The price of a ticket to {destination_city} is {price}"

# *** End - First Custom Funtion (Tool) ***


# *** Begin - Second Custom Funtion (Tool) ***

# Dictionary with ticket price to different cities - we can modify this to find from a DB table
ticket_info = {"london": "L-0158", "paris": "P-9419", "tokyo": "T-8821", "berlin": "B-5219"}

# Function to get the ticket price of a given city
def get_ticket_info(destination_city):
    # The following print statement is optional to show when tool calling is happening in code output
    print(f"Booking information tool called for city {destination_city}")
    booking_number = ticket_info.get(destination_city.lower(), "No booking info available")
    return f"The price of a ticket to {destination_city} is {booking_number}"

# *** End - Second Custom Funtion (Tool) ***


# *** Begin - Create a JSON, which will tell the LLM about what tool needs to be called in which condition.***

# Dictionary for the first tool function
price_function = {
    "name": "get_ticket_price", # The name of the custom function (tool)
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": { # Define each parameters required by the tool function, we have only 1 called destination_city
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string", # Data type of the input parameter
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

# Dictionary for the second tool function
booking_function = {
    "name": "get_ticket_info", # The name of the custom function (tool)
    "description": "Get the information of ticket booked to the destination city.",
    "parameters": { # Define each parameters required by the tool function, we have only 1 called destination_city
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string", # Data type of the input parameter
                "description": "The city that the customer has booked ticket to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

# Create JSON with details of both the tools
tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": booking_function}
]

# *** End - Create a JSON, which will tell the LLM about what tool needs to be called in which condition.***


# *** Begin - The function that parses the response fro first LLM call, call the tool function and get response from it

def handle_tool_call(message):
    response = []

    for tool_call in message.tool_calls:

        # Check for the first tool call
        if tool_call.function.name == "get_ticket_price":
            # Parse the arguments for the tool function from the message
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            # Call the tool function
            price_details = get_ticket_price(city)
            # Generate the response in following JSON format
            response.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })

        # Check for the second tool call
        if tool_call.function.name == "get_ticket_info":
            # Parse the arguments for the tool function from the message
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            # Call the tool function
            booking_details = get_ticket_info(city)
            # Generate the response in following JSON format
            response.append({
                "role": "tool",
                "content": booking_details,
                "tool_call_id": tool_call.id
            })

    return response

# *** End - The function that parses the response fro first LLM call, call the tool function and get response from it

# Define the chat function
def chatbot_with_tool(user_prompt, context_history):
    
    # Instantiate OpenAI with proper API key
    model_api = OpenAI(api_key=OPENAI_API_KEY, base_url=OPENAI_BASE_URL)
    model="gpt-4.1-mini"

    # Extract 'role' and 'context' values from the conversation history and build a dictionary
    history = [{"role":h["role"], "content":h["content"]} for h in context_history]
    # Create the new message list with system prompt, conversation history and the new user prompt
    messages=[{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": user_prompt}]

    # Make the FIRST call to the LLM with the system prompt, user prompt, and the chat history
    response = model_api.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools # This is the new parameter with all the info of custom function (tool) to the LLM
    )

    # Evaluate if the LLM response indicated any need of tool call
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        tool_responses = handle_tool_call(message)
        messages.append(message)
        messages.extend(tool_responses)

        # Make the SECOND call to the LLM with the entire message thread combined with the response from the tool
        # Note that this time we are not passing the third parameter 'tools=tools', because we just want the LLM
        # to summarise the entire message and the response from tool and generate a nice output to the user.
        response = model_api.chat.completions.create(model=model, messages=messages)
    else:
        # The following print statement is optional to show when tool calling is happening in code output
        print("No tool called yet")



    # Return the output generated by the LLM
    return response.choices[0].message.content


# Instantiate gradio interface
interface = gr.ChatInterface(fn=chatbot_with_tool, type="messages", title="ChatBot with Multiple Tools")

interface.launch()